# Mouse and Cheese Reinforcement Learning Game using Q-table

## Karol Pysiak https://github.com/pysiakk

### General idea

This is my first project in the RL area. There I'm using Q-table agent method. Q-table is a table containing expected rewards after taking different actions. There is one row for every state in our model and in every row there is column for every action that we can do. To tune Q-table we use the Bellman equation.
$$
NewQ(s,a) = Q(s,a) + \alpha[R(s,a) + \gamma \max\limits_{a'}Q(s',a') - Q(s,a)]
$$
Where $\alpha$ is a learning rate, $\gamma$ is a discount rate, $R$ is Reward table, $s'$ is a next state and $a'$ are actions possible in state $s'$. This project was inspired by article "Diving deeper into Reinforcement Learning with Q-Learning" written by Thomas Simonini: https://www.freecodecamp.org/news/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe/ . All information that I needed to write this project I found in this article.

### Rules of the game

Firstly, we are preparing a kind of a maze for a mouse. There are three types of fields: cheese, traps and paths. One episode of the game ends when mouse finds the cheese or walks into a trap. For finding cheese mouse is gaining a lot of points and losing for being killed. We can place many traps and many pieces of cheese and give them different importance. Let's assume that trap has $<<-1$ points and cheese has $>>-1$ points. Now the challage is to eat the optimal cheese and not being killed by the trap.

## Code

### Imports

In [57]:
from random import random, randint, sample
from numpy import argmax, zeros, ones
from tabulate import tabulate

### Update Q-table

There we use the Bellman equation to update Q-table. Quite easy, right?

In [58]:
def update(state, state_new, action, Q_table, Reward, alpha, gamma):
    
    Q_table[state][action] = Q_table[state][action] + (alpha *
        (Reward[state_new] + gamma * max(Q_table[state_new]) - Q_table[state][action]))
    
    return Q_table

### Non-zero function

I created it for the purpose of cleaner code in the next functions.

In [59]:
def non_zero(input):
    
    output = []
    
    for i in range(len(input)):
        if input[i] != 0:
            output.append(i)
    
    return output

### Let's move

Here we choose which action should we choose, where should we go. To avoid being stuck in a loop we created the `epsilon`. It tells us how many steps we should take randomly and how much should we decide by using `Q_table`. After deciding which step we are taking, we convert out action into the state that we are going into and we update `Q_table`.

In [60]:
def move(epsilon, state, Q_table, Reward, alpha, gamma, Moves, dim):
    
    state_new = state
    # There I created a little matrix that in a first row has field that we can walk onto
    # and in a second row there are values from Q_table that refer to these fields
    nz = [non_zero(Moves[state]), [Q_table[state][i] for i in non_zero(Moves[state])]]
    
    if random() < epsilon: # with the probability equal to epsilon we choose our action randomly
        action = sample(nz[0], 1)[0]    
    else:
        action = nz[0][argmax(nz[1])] # we simply choose the path with the greatest expected reward
        
    if action == 0:                # move left
        state_new = state - 1
    elif action == 1:              # move right
        state_new = state + 1
    elif action == 2:              # move up
        state_new = state - dim[1]
    elif action == 3:              # move down
        state_new = state + dim[1]

    Q_table = update(state, state_new, action, Q_table, Reward, alpha, gamma)
        
    return state_new, Q_table

### PrintMap

Only purpose of this function is to look how our mouse is moving. It's cool to watch, but really slows down the process.

In [61]:
def printMap(state, Q_table, Reward, dim):
    
    for j in range(len(Q_table)):
        print('[', end='')
        
        if j == state:
            print('X', end='')
        elif Reward[j] < -1:
            print('T', end='')
        elif Reward[j] > -1:
            print('C', end='')
        else:
            print(' ', end='')

        print(']', end='')

        if (j%dim[1]) == (dim[1]-1):
            print('\n', end='')

        print('\n')

### One episode

Episode begins when we place our mouse in the maze and ends when it either eats or dies. This function is resposible for controlling what mouse is really doing. Besides the `Q_table` it returns `response` which is encoded in this way:

* $1$ - mouse found the cheese (episode ended)
* $0$ - mouse stepped onto a normal field (episode continues)
* $-1$ - mouse went into the trap (episode ended)

Almost whole function runs in a loop until mouse ends the episode.

In [62]:
def learnEpisode(epsilon, start_state, Q_table, Reward, alpha, gamma, Moves, dim, prt):
    
    state = start_state
    
    if prt:
        printMap(state, Q_table, Reward, dim)
    
    response = 0
    
    while response == 0:     # loops stops when the mouse ends the episode 
                             # and this happens when the response is 1 or -1
        
        state, Q_table = move(epsilon, state, Q_table, Reward, alpha, gamma, Moves, dim)
        
        if prt:
            printMap(state, Q_table, Reward, dim)
        
        if Reward[state] > -1:
            response = 1             # mouse found the cheese so the response is 1
        elif Reward[state] < -1:
            response = -1            # mouse walked into the trap so the response is -1
        else:
            response = 0             # mouse is walking the neutral path, response is 0
        
    return response, Q_table

### Headquarter

There we initiate and control the whole process of learning. Firstly, we create `Q_table` filled up with zeros and `Moves` table where you can find all the moves that mouse can do from any state (1 move possible, 0 move impossible). We run learning in epochs. There every epoch consists of many learning episodes, one starting from every field that is neutral. After every epoch we decrease the `epsilon` to move more using the knowledge that we acquired in previous epochs. 

In [63]:
def learn(Reward, alpha, gamma, n_epochs, dim, prt, epsilon_rate):
    
    epsilon = 1    # we set epsilon to 0 because at the begining 
                   # we lack the knowledge so we try choosing random actions
    count = 0
    # we initialize Q_table with zeros, because with have zero knowledge
    Q_table = [list(zeros(4, dtype=int)) for i in range(dim[0]*dim[1])]
    # to build our movement map we firstly initalize matrix of ones
    Moves = [list(ones(4, dtype=int)) for i in range(dim[0]*dim[1])]
    
    # then at fields connected with walls we change appropriate action possibilities to zero
    for i in range(len(Moves)):
        if i//dim[1] == 0:         # top of the box
            Moves[i][2] = 0
        if i//dim[1] == dim[0]-1:  # bottom of the box
            Moves[i][3] = 0
        if i%dim[1] == 0:          # left side of the box
            Moves[i][0] = 0
        if i%dim[1] == dim[1]-1:   # right side of the box
            Moves[i][1] = 0 
            
    if dim[0]*dim[1] != len(Reward):     # checking if reward table is valid
        print("Length of Reward table and dim of matrix don't match")
    
    for epoch in range(n_epochs):
        # we choose only netural fields to start our mouse's walk from
        starting_states = list(set(range(len(Reward))) - set(non_zero([i+1 for i in Reward])))
        
        for start_state in starting_states:
            response, Q_table = learnEpisode(epsilon, start_state, Q_table, Reward, 
                                             alpha, gamma, Moves, dim, prt)
            
            if response == 1:
                count += 1
                
            print('epoch: {}, acc: {:.2f}%'
                  .format(epoch+1, count/
                          ((epoch*len(starting_states)) + starting_states.
                           index(start_state) + 1)*100) +
                  '\n\n==========================================================')
        
        epsilon = epsilon * (1 - epsilon_rate)
    
    return Q_table    # we return Q_table because it is the most valuable information after learning
                      # this is what we have really learnt

### Hyperparameters

In [64]:
dim = [4, 4] # shape of our box
# reward vector for every field (by default neutral field is -1, cheese is >> -1 and trap is << -1)
Reward = [-1, -1, -1, 5, -1, -10, -1, -1, -1, -1, -10, -1, -10, -1, 10, -1] 
gamma = 0.9 # discount rate
alpha = 0.1 # learning rate
epsilon = 1 # starting random steps threshold
epsilon_rate = 0.01 # rate of decreasing random steps threshold
n_epochs = 1000 # number of epochs
prt = False # do we want to print the maze after each step

### Exmaple usage

In [65]:
Q = learn(Reward, alpha, gamma, n_epochs, dim, prt, epsilon_rate)

epoch: 1, acc: 0.00%

epoch: 1, acc: 50.00%

epoch: 1, acc: 33.33%

epoch: 1, acc: 25.00%

epoch: 1, acc: 20.00%

epoch: 1, acc: 33.33%

epoch: 1, acc: 28.57%

epoch: 1, acc: 25.00%

epoch: 1, acc: 33.33%

epoch: 1, acc: 30.00%

epoch: 1, acc: 27.27%

epoch: 2, acc: 33.33%

epoch: 2, acc: 30.77%

epoch: 2, acc: 35.71%

epoch: 2, acc: 33.33%

epoch: 2, acc: 31.25%

epoch: 2, acc: 35.29%

epoch: 2, acc: 33.33%

epoch: 2, acc: 31.58%

epoch: 2, acc: 30.00%

epoch: 2, acc: 28.57%

epoch: 2, acc: 31.82%

epoch: 3, acc: 34.78%

epoch: 3, acc: 33.33%

epoch: 3, acc: 32.00%

epoch: 3, acc: 30.77%

epoch: 3, acc: 29.63%

epoch: 3, acc: 32.14%

epoch: 3, acc: 31.03%

epoch: 3, acc: 30.00%

epoch: 3, acc: 29.03%

epoch: 3, acc: 28.12%

epoch: 3, acc: 30.30%

epoch: 4, acc: 29.41%

epoch: 4, acc: 28.57%

epoch: 4, acc: 27.78%

epoch: 4, acc: 27.03%

epoch: 4, acc: 26.32%

epoch: 4, acc: 25.64%

epoch: 4, acc: 25.00%

epoch: 4, acc: 24.39%

epoch: 4, acc: 26.19%

epoch: 4, acc: 27.91%

epoch: 4, ac

epoch: 152, acc: 63.96%

epoch: 152, acc: 63.98%

epoch: 152, acc: 64.00%

epoch: 152, acc: 64.02%

epoch: 152, acc: 64.05%

epoch: 152, acc: 64.01%

epoch: 152, acc: 64.03%

epoch: 152, acc: 64.05%

epoch: 152, acc: 64.07%

epoch: 152, acc: 64.09%

epoch: 152, acc: 64.11%

epoch: 153, acc: 64.14%

epoch: 153, acc: 64.16%

epoch: 153, acc: 64.18%

epoch: 153, acc: 64.20%

epoch: 153, acc: 64.16%

epoch: 153, acc: 64.12%

epoch: 153, acc: 64.15%

epoch: 153, acc: 64.11%

epoch: 153, acc: 64.13%

epoch: 153, acc: 64.15%

epoch: 153, acc: 64.17%

epoch: 154, acc: 64.19%

epoch: 154, acc: 64.21%

epoch: 154, acc: 64.23%

epoch: 154, acc: 64.26%

epoch: 154, acc: 64.22%

epoch: 154, acc: 64.24%

epoch: 154, acc: 64.26%

epoch: 154, acc: 64.28%

epoch: 154, acc: 64.30%

epoch: 154, acc: 64.32%

epoch: 154, acc: 64.34%

epoch: 155, acc: 64.31%

epoch: 155, acc: 64.33%

epoch: 155, acc: 64.35%

epoch: 155, acc: 64.31%

epoch: 155, acc: 64.33%

epoch: 155, acc: 64.35%

epoch: 155, acc: 64.37%



epoch: 269, acc: 75.61%

epoch: 269, acc: 75.62%

epoch: 269, acc: 75.63%

epoch: 269, acc: 75.63%

epoch: 270, acc: 75.61%

epoch: 270, acc: 75.62%

epoch: 270, acc: 75.62%

epoch: 270, acc: 75.63%

epoch: 270, acc: 75.64%

epoch: 270, acc: 75.65%

epoch: 270, acc: 75.66%

epoch: 270, acc: 75.67%

epoch: 270, acc: 75.67%

epoch: 270, acc: 75.68%

epoch: 270, acc: 75.69%

epoch: 271, acc: 75.66%

epoch: 271, acc: 75.67%

epoch: 271, acc: 75.68%

epoch: 271, acc: 75.69%

epoch: 271, acc: 75.70%

epoch: 271, acc: 75.71%

epoch: 271, acc: 75.71%

epoch: 271, acc: 75.72%

epoch: 271, acc: 75.73%

epoch: 271, acc: 75.74%

epoch: 271, acc: 75.75%

epoch: 272, acc: 75.75%

epoch: 272, acc: 75.76%

epoch: 272, acc: 75.77%

epoch: 272, acc: 75.78%

epoch: 272, acc: 75.79%

epoch: 272, acc: 75.80%

epoch: 272, acc: 75.80%

epoch: 272, acc: 75.81%

epoch: 272, acc: 75.82%

epoch: 272, acc: 75.83%

epoch: 272, acc: 75.84%

epoch: 273, acc: 75.84%

epoch: 273, acc: 75.85%

epoch: 273, acc: 75.86%



epoch: 404, acc: 82.79%

epoch: 404, acc: 82.80%

epoch: 404, acc: 82.80%

epoch: 404, acc: 82.80%

epoch: 404, acc: 82.81%

epoch: 405, acc: 82.81%

epoch: 405, acc: 82.82%

epoch: 405, acc: 82.82%

epoch: 405, acc: 82.82%

epoch: 405, acc: 82.83%

epoch: 405, acc: 82.83%

epoch: 405, acc: 82.84%

epoch: 405, acc: 82.84%

epoch: 405, acc: 82.84%

epoch: 405, acc: 82.85%

epoch: 405, acc: 82.85%

epoch: 406, acc: 82.85%

epoch: 406, acc: 82.86%

epoch: 406, acc: 82.86%

epoch: 406, acc: 82.87%

epoch: 406, acc: 82.87%

epoch: 406, acc: 82.87%

epoch: 406, acc: 82.88%

epoch: 406, acc: 82.88%

epoch: 406, acc: 82.89%

epoch: 406, acc: 82.89%

epoch: 406, acc: 82.89%

epoch: 407, acc: 82.90%

epoch: 407, acc: 82.90%

epoch: 407, acc: 82.90%

epoch: 407, acc: 82.91%

epoch: 407, acc: 82.91%

epoch: 407, acc: 82.92%

epoch: 407, acc: 82.92%

epoch: 407, acc: 82.92%

epoch: 407, acc: 82.93%

epoch: 407, acc: 82.93%

epoch: 407, acc: 82.94%

epoch: 408, acc: 82.94%

epoch: 408, acc: 82.94%



epoch: 529, acc: 86.76%

epoch: 529, acc: 86.76%

epoch: 529, acc: 86.77%

epoch: 529, acc: 86.77%

epoch: 529, acc: 86.77%

epoch: 529, acc: 86.77%

epoch: 529, acc: 86.78%

epoch: 529, acc: 86.78%

epoch: 529, acc: 86.78%

epoch: 529, acc: 86.78%

epoch: 529, acc: 86.78%

epoch: 530, acc: 86.79%

epoch: 530, acc: 86.79%

epoch: 530, acc: 86.79%

epoch: 530, acc: 86.79%

epoch: 530, acc: 86.80%

epoch: 530, acc: 86.80%

epoch: 530, acc: 86.80%

epoch: 530, acc: 86.80%

epoch: 530, acc: 86.81%

epoch: 530, acc: 86.81%

epoch: 530, acc: 86.81%

epoch: 531, acc: 86.81%

epoch: 531, acc: 86.81%

epoch: 531, acc: 86.82%

epoch: 531, acc: 86.82%

epoch: 531, acc: 86.82%

epoch: 531, acc: 86.82%

epoch: 531, acc: 86.83%

epoch: 531, acc: 86.83%

epoch: 531, acc: 86.83%

epoch: 531, acc: 86.83%

epoch: 531, acc: 86.83%

epoch: 532, acc: 86.84%

epoch: 532, acc: 86.84%

epoch: 532, acc: 86.84%

epoch: 532, acc: 86.84%

epoch: 532, acc: 86.85%

epoch: 532, acc: 86.85%

epoch: 532, acc: 86.85%



epoch: 666, acc: 89.46%

epoch: 666, acc: 89.46%

epoch: 666, acc: 89.46%

epoch: 666, acc: 89.46%

epoch: 666, acc: 89.46%

epoch: 667, acc: 89.46%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.47%

epoch: 667, acc: 89.48%

epoch: 667, acc: 89.48%

epoch: 667, acc: 89.48%

epoch: 668, acc: 89.48%

epoch: 668, acc: 89.48%

epoch: 668, acc: 89.48%

epoch: 668, acc: 89.48%

epoch: 668, acc: 89.49%

epoch: 668, acc: 89.49%

epoch: 668, acc: 89.49%

epoch: 668, acc: 89.49%

epoch: 668, acc: 89.49%

epoch: 668, acc: 89.49%

epoch: 668, acc: 89.49%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.50%

epoch: 669, acc: 89.51%

epoch: 669, acc: 89.51%

epoch: 669, acc: 89.51%

epoch: 669, acc: 89.51%

epoch: 670, acc: 89.51%

epoch: 670, acc: 89.51%



epoch: 792, acc: 91.13%

epoch: 792, acc: 91.13%

epoch: 792, acc: 91.13%

epoch: 792, acc: 91.14%

epoch: 792, acc: 91.14%

epoch: 792, acc: 91.14%

epoch: 792, acc: 91.14%

epoch: 793, acc: 91.14%

epoch: 793, acc: 91.14%

epoch: 793, acc: 91.14%

epoch: 793, acc: 91.14%

epoch: 793, acc: 91.14%

epoch: 793, acc: 91.14%

epoch: 793, acc: 91.15%

epoch: 793, acc: 91.15%

epoch: 793, acc: 91.15%

epoch: 793, acc: 91.15%

epoch: 793, acc: 91.15%

epoch: 794, acc: 91.15%

epoch: 794, acc: 91.15%

epoch: 794, acc: 91.15%

epoch: 794, acc: 91.15%

epoch: 794, acc: 91.15%

epoch: 794, acc: 91.16%

epoch: 794, acc: 91.16%

epoch: 794, acc: 91.16%

epoch: 794, acc: 91.16%

epoch: 794, acc: 91.16%

epoch: 794, acc: 91.16%

epoch: 795, acc: 91.16%

epoch: 795, acc: 91.16%

epoch: 795, acc: 91.16%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%

epoch: 795, acc: 91.17%



epoch: 920, acc: 92.37%

epoch: 920, acc: 92.37%

epoch: 920, acc: 92.37%

epoch: 921, acc: 92.37%

epoch: 921, acc: 92.37%

epoch: 921, acc: 92.37%

epoch: 921, acc: 92.37%

epoch: 921, acc: 92.38%

epoch: 921, acc: 92.38%

epoch: 921, acc: 92.38%

epoch: 921, acc: 92.38%

epoch: 921, acc: 92.38%

epoch: 921, acc: 92.38%

epoch: 921, acc: 92.38%

epoch: 922, acc: 92.38%

epoch: 922, acc: 92.38%

epoch: 922, acc: 92.38%

epoch: 922, acc: 92.38%

epoch: 922, acc: 92.38%

epoch: 922, acc: 92.38%

epoch: 922, acc: 92.39%

epoch: 922, acc: 92.39%

epoch: 922, acc: 92.39%

epoch: 922, acc: 92.39%

epoch: 922, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.39%

epoch: 923, acc: 92.40%

epoch: 923, acc: 92.40%

epoch: 924, acc: 92.40%

epoch: 924, acc: 92.40%

epoch: 924, acc: 92.40%

epoch: 924, acc: 92.40%



### Pretty way of printing our Q-table

In [66]:
print(tabulate([[i] + Q[i] for i in range(len(Q))], headers =['Left', 'Right', 'Up', 'Down'], 
               tablefmt='orgtbl'))

|    |     Left |    Right |       Up |     Down |
|----+----------+----------+----------+----------|
|  0 |  0       |  2.14992 |  0       |  3.122   |
|  1 |  1.60343 |  3.5     |  0       | -9.997   |
|  2 |  2.14832 |  5       |  0       |  3.09833 |
|  3 |  0       |  0       |  0       |  0       |
|  4 |  0       | -9.99803 |  1.74022 |  4.58    |
|  5 |  0       |  0       |  0       |  0       |
|  6 | -9.96619 |  4.58    |  3.49975 | -9.96243 |
|  7 |  3.0933  |  0       |  4.99969 |  6.2     |
|  8 |  0       |  6.2     |  3.11994 | -9.99999 |
|  9 |  4.573   | -9.9995  | -9.9963  |  8       |
| 10 |  0       |  0       |  0       |  0       |
| 11 | -9.99976 |  0       |  4.57852 |  8       |
| 12 |  0       |  0       |  0       |  0       |
| 13 | -9.9999  | 10       |  6.19999 |  0       |
| 14 |  0       |  0       |  0       |  0       |
| 15 | 10       |  0       |  6.19995 |  0       |


## Summary

Hope you enjoyed this little ride. This project gave me so much fun. It is a great way to jump into Reinforcement Learning world. Q-table learning is one of the basic RL algorithms, but it gives a great insight into how this kind od Machine Learning works. I keep in mind that this implementation can be imperfect yet, so feel free to test it out and if you find any bugs just create and issue at my repository. And one more time, I extremely recommend "Deep Reinforcement Learning Course" by Thomas Simonini which you can find here: https://simoninithomas.github.io/Deep_reinforcement_learning_Course/ . If you are new to Reinforcement Learning you will surely find there something that will help you start your marvelous adventrure.